In [1]:
from pathlib import Path

import torch

from src.models.feature_models import NonAttentiveTacotron, NonAttentiveTacotronVoicePrint
from src.constants import CHECKPOINT_DIR, FEATURE_CHECKPOINT_NAME, FEATURE_MODEL_FILENAME, MELS_MEAN_FILENAME, MELS_STD_FILENAME, REFERENCE_PATH

In [2]:
CHECKPOINT_NAME = "test"
DEVICE = "cuda"

In [3]:
checkpoint_path = CHECKPOINT_DIR / CHECKPOINT_NAME / FEATURE_CHECKPOINT_NAME

In [4]:
model = torch.load(checkpoint_path / FEATURE_MODEL_FILENAME, map_location=DEVICE)

In [5]:
model.eval();

In [6]:
gst = model.gst

In [7]:
mels_std = torch.load(checkpoint_path / MELS_STD_FILENAME, map_location=DEVICE)
mels_mean = torch.load(checkpoint_path / MELS_MEAN_FILENAME, map_location=DEVICE)

In [8]:
result = {}
for speaker in REFERENCE_PATH.iterdir():
    for emo in speaker.iterdir():
        ref_mel = torch.load(emo, map_location=DEVICE)
        ref_mel = (ref_mel - mels_mean) / mels_std
        emb = gst(ref_mel.unsqueeze(0).permute(0, 2, 1).float())
        result[f"{speaker.name}_{emo.stem}"] = emb.squeeze()